# Numeric

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('IMDB-Movie-Data.csv')
df = df.drop(df.columns[[2,3,4,5,6,7,10]],axis = 1)
df = df.dropna(axis=0)

import MICE

X = df.iloc[:, [3,4]].values  
y = df.iloc[:, 2].values

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

#actual
actual=np.concatenate((y_train,y_test),axis=0)
r1=len(y_train)

r=len(y_test)
y_test_n=y_test.copy()
y_test_n[0:r]=np.nan

y_train_1=np.concatenate((y_train,y_test_n),axis=0)
X_train_1=np.concatenate((X_train,X_test),axis=0)

df_2 = pd.DataFrame({'Rating':y_train_1[:],'Votes':X_train_1[:,0],'Metascore':X_train_1[:,1]})



# MICE

In [10]:
mice_result = MICE.MICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

print(mice_result.shape)
df_3 = pd.DataFrame(mice_result, columns = ['Rating', 'Votes', 'Metascore'])
predict = df_3['Rating']

predict = np.array(predict)

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(actual[r1:], predict[r1:]))  
print('Mean Squared Error:', metrics.mean_squared_error(actual[r1:], predict[r1:]))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test,predict[r1:]))) 

(936, 3)
Mean Absolute Error: 0.95
Mean Squared Error: 1.5419148936170213
Root Mean Squared Error: 1.2417386575350795


# iMICE

In [11]:
import iMICE

imice_result = iMICE.iMICE(verbose=False, init_fill_method="median", impute_type="pmm", n_imputations=7).complete(np.matrix(df_2))

df_3i = pd.DataFrame(imice_result, columns = ['Age', 'Pclass', 'Fare'])
predicti = df_3i['Age']

predicti = np.array(predicti)

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(actual[r1:], predicti[r1:]))  
print('Mean Squared Error:', metrics.mean_squared_error(actual[r1:], predicti[r1:]))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predicti[r1:]))) 

Mean Absolute Error: 0.7335106382978722
Mean Squared Error: 0.881968085106383
Root Mean Squared Error: 0.9391315589981964


# Random Forest

In [12]:
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
regressor = RandomForestRegressor(n_estimators=20, random_state=0)  
regressor.fit(X_train, y_train)  
y_pred = regressor.predict(X_test) 

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.5713829787234043
Mean Squared Error: 0.5452324468085106
Root Mean Squared Error: 0.738398569072632


# SVR

In [13]:
from sklearn.svm import SVR
svr = SVR(gamma=0.001, C=1.0, epsilon=0.2)
svr.fit(X_train, y_train)
y_pred=svr.predict(X_test)

from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.7076592330661619
Mean Squared Error: 0.7613599641251612
Root Mean Squared Error: 0.8725594330045152
